# Simulace Klient-Server v jednom notebooku

Běžně musíme pro testování síťových aplikací otevřít dvě terminálová okna (jedno pro server, jedno pro klienta). 

Jak jsme se naučili v **sekci 17 (Multiprocessing)**, díky modulu `multiprocessing` to můžeme simulovat přímo v Python skriptu nebo Jupyter Notebooku. Vytvoříme dva procesy:
1. **Server Proces:** Nastartuje se, začne naslouchat a čekat.
2. **Klient Proces:** Po chvilce se nastartuje, připojí se k serveru a pošle data.

Tento přístup je skvělý pro automatizované testování sítí. Procesy jsou zde nezbytné, protože server v `while True` cyklu by v jednom procesu zablokoval zbytek kódu.

In [ ]:
import socket
import multiprocessing
import time
import os

# Konfigurace
HOST = '127.0.0.1'
PORT = 9999

## 1. Definice Serveru
Funkce, která poběží v prvním procesu. Všimněte si, že používáme `print(..., flush=True)`, abychom v Jupyteru viděli výstup okamžitě.

In [ ]:
def run_server():
    pid = os.getpid()
    print(f"[Server {pid}] Startuji na {HOST}:{PORT}...", flush=True)
    
    # Vytvoření socketu
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        # Povolíme okamžitý restart portu (viz README - SO_REUSEADDR)
        s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        s.bind((HOST, PORT))
        s.listen()
        
        print(f"[Server {pid}] Čekám na připojení...", flush=True)
        
        # Blokující čekání na klienta
        conn, addr = s.accept()
        
        with conn:
            print(f"[Server {pid}] Klient připojen: {addr}", flush=True)
            while True:
                data = conn.recv(1024)
                if not data:
                    break
                
                zprava = data.decode('utf-8')
                print(f"[Server {pid}] Přijato: '{zprava}'", flush=True)
                
                # Odpověď (Echo)
                conn.sendall(f"Potvrzuji příjem: {zprava}".encode('utf-8'))
                
    print(f"[Server {pid}] Konec.", flush=True)

## 2. Definice Klienta
Funkce pro druhý proces. Simuluje uživatele.

In [ ]:
def run_client(zpravy):
    pid = os.getpid()
    # Malá pauza, aby server stihl nastartovat a provést bind()
    time.sleep(1) 
    print(f"[Klient {pid}] Připojuji se...", flush=True)
    
    try:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            s.connect((HOST, PORT))
            
            for zprava in zpravy:
                print(f"[Klient {pid}] Odesílám: {zprava}", flush=True)
                s.sendall(zprava.encode('utf-8'))
                
                odpoved = s.recv(1024)
                print(f"[Klient {pid}] Server odpověděl: {odpoved.decode('utf-8')}", flush=True)
                time.sleep(0.5)
                
    except ConnectionRefusedError:
        print(f"[Klient {pid}] CHYBA: Server neběží!", flush=True)

## 3. Spuštění simulace
Zde použijeme `multiprocessing.Process` pro spuštění obou funkcí najednou.

In [ ]:
if __name__ == "__main__":
    print("--- START SIMULACE ---")
    
    # 1. Vytvoření procesu pro server
    p_server = multiprocessing.Process(target=run_server)
    
    # 2. Vytvoření procesu pro klienta
    testovaci_data = ["Ahoj", "Python je super", "Multiprocessing funguje"]
    p_client = multiprocessing.Process(target=run_client, args=(testovaci_data,))
    
    # 3. Spuštění
    p_server.start()
    p_client.start()
    
    # 4. Čekání na dokončení klienta
    p_client.join()
    
    # 5. Ukončení serveru
    if p_server.is_alive():
        print("--- Ukončuji server ---")
        p_server.terminate()
        p_server.join()
        
    print("--- HOTOVO ---")

## 4. Úkol pro studenty

Zkuste upravit funkci `run_server` tak, aby:
1. Převáděla přijatý text na **velká písmena** (UPPERCASE).
2. Počítala, kolik zpráv už od daného klienta přijala, a toto číslo posílala v odpovědi zpět.